## Чистим русские диалоги

In [1]:
import nltk
from nltk.collocations import *
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from collections import Counter

import base64
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
from wordcloud import WordCloud

import pymorphy2
from tqdm import tqdm
import copy
import os
import random

In [2]:
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']

st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
      "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
alphabet = 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя'

In [335]:
repeat_phr = ['( )+',
 '(эй )+',
 '(э )+',
 '(ха )+',
 '(размытие )+',
 '(ох )+',
 '(хо )+',
 '(ой )+',
 '(подожди )+',
 '(подождите )+',
 '(ладно )+',
 '(ла )+',
 '(смотри )+',
 '(смотрите )+',
 '(ву )+',
 '(бла )+',
 '(хур )+',
 '(хорошо )+',
 '(погоди )+',
 '(погодите )+',
 '(ага )+',
 '(давай )+',
 '(хорошая игра )+',
 '(вонючий кот )+',
 '(7 )+',
 '(друг )+',
 '(я )+',
 '(нет )+']
def delete_rep(text):
    for ban in repeat_phr:
        text = re.sub(ban, ban[1:-2], text)
    return text

banned_phr = [('Погодите-подождите', "Погодите"), ('Я-я', "Я"), ("это -Это", "это-это")]
def delete_phr(text):
    for ban in banned_phr:
        text = re.sub(ban[0], ban[1], text)
    return text

banned_bigrams = ['(я хочу )+', 
    '(я хотела )+', 
    '(я хотел )+', 
    '(мы хотим )+', 
    '(мы хотели )+', 
    '(я не хочу )+', 
    '(я не хотела )+', 
    '(я не хотел )+', 
    '(мы не хотим )+', 
    '(мы не хотели )+', 
    '(я хочу, чтобы )+', 
    '(я хотела, чтобы )+', 
    '(я хотел, чтобы )+', 
    '(мы хотим, чтобы )+', 
    '(мы хотели, чтобы )+', 
    '(я не хочу, чтобы )+', 
    '(я не хотела, чтобы )+', 
    '(я не хотел, чтобы )+', 
    '(мы не хотим, чтобы )+', 
    '(мы не хотели, чтобы )+', 
    '(я думаю )+', 
    '(я думал )+', 
    '(я думала )+', 
    '(мы думаем )+', 
    '(мы думали )+', 
    '(я не думаю )+', 
    '(я не думал )+', 
    '(я не думала )+', 
    '(мы не думаем )+', 
    '(мы не думали )+', 
    '(я думаю, что )+', 
    '(я думал, что )+', 
    '(я думала, что )+', 
    '(мы думаем, что )+', 
    '(мы думали, что )+', 
    '(я не думаю, что )+', 
    '(я не думал, что )+', 
    '(я не думала, что )+', 
    '(мы не думаем, что )+', 
    '(мы не думали, что )+', 
    '(я знаю )+', 
    '(я знал )+', 
    '(я знала )+', 
    '(мы знаем )+', 
    '(мы знали )+', 
    '(я не знаю )+', 
    '(я не знал )+', 
    '(я не знала )+', 
    '(мы не знаем )+', 
    '(мы не знали )+', 
    '(я знаю, что )+', 
    '(я знал, что )+', 
    '(я знала, что )+', 
    '(мы знаем, что )+', 
    '(мы знали, что )+', 
    '(я не знаю, что )+', 
    '(я не знал, что )+', 
    '(я не знала, что )+', 
    '(мы не знаем, что )+', 
    '(мы не знали, что )+', 
    '(я имею в виду )+', 
    '(я имела в виду )+', 
    '(я имел в виду )+', 
    '(мы имеем в виду )+', 
    '(мы имели в виду )+', 
    '(я не имею в виду )+', 
    '(я не имела в виду )+', 
    '(я не имел в виду )+', 
    '(мы не имеем в виду )+', 
    '(мы не имели в виду )+', 
    '(я имею в виду, что )+', 
    '(я имела в виду, что )+', 
    '(я имел в виду, что )+', 
    '(мы имеем в виду, что )+', 
    '(мы имели в виду, что )+', 
    '(я не имею в виду, что )+', 
    '(я не имела в виду, что )+', 
    '(я не имел в виду, что )+', 
    '(мы не имеем в виду, что )+', 
    '(мы не имели в виду, что )+', 
    '(я могу )+', 
    '(я мог )+', 
    '(я могла )+', 
    '(мы можем )+', 
    '(мы могли )+', 
    '(я могу бы )+', 
    '(я мог )+', 
    '(я могла )+', 
    '(мы можем )+', 
    '(мы могли )+', 
    '(я могу )+', 
    '(я мог )+', 
    '(я могла )+', 
    '(мы можем )+', 
    '(мы могли )+']
def delete_trash(text):
    for ban in banned_bigrams:
        text = re.sub(ban, '', text)  
    for ban in banned_bigrams:
        text = re.sub(ban[0]+ban[1].upper()+ban[2:], '', text)  
    for ban in banned_bigrams:
        text = re.sub(ban[:-3] + ',' + ban[-2:], '', text)  
    for ban in banned_bigrams:
        text = re.sub(ban[0]+ban[1].upper()+ban[2:-3] + ',' + ban[-2:], '', text)  
    return text

beginnings = [('года. МОНИКА', 'МОНИКА'), 
              ('ЧЕНД ЛЕР','ЧЕНДЛЕР'),
              ('человека. ЧЕНДЛЕР','ЧЕНДЛЕР'),
              ('ФИБ И','ФИБИ'),
              ('МО. НИКА','МОНИКА'),
              ('Росс. МОНИКА','МОНИКА'),
              ('ФИБИ Фиби','ФИБИ'),
              ('Тома и Сью НЕДРУГ','НЕДРУГ'),
              ('Чендлера ЧЕНДЛЕР','ЧЕНДЛЕР'),
              ('Ч ЕНДЛЕР','d')]
beginnings = dict(beginnings)

In [389]:
# чистим от лишних символов, повторяющихся символов и слов
scripts = {i : {} for i in range(1, 11)}

for season in tqdm(range(1, 11)):
    for serie in tqdm(range(1,30), desc='\t'):
        url = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/main/data/scripts/russian/{season}/{serie}.txt'
        req = requests.get(url)
        if req.status_code == requests.codes.ok:
            text1 = []
            req = BeautifulSoup(req.text, 'html.parser')
            text = str(req)
            text = re.sub("5923", "", text)
            text = text.split('\n')
            for el in text:
                el1 = el
                el = delete_rep(el)
                el = delete_phr(el)
                el = re.sub("№", "Нет", el)  
                el = re.sub("No", "Нет", el)  
                el = re.sub("(no)+", "нет", el)  

                for i in ("\.", "\?", ",", "!"):
                    el = re.sub(f" {i}", f"{i[-1]}", el)
                el = re.sub("<(.*?)>", "", el)
                for i in sorted(alphabet)[33:]:
                    pattern = f"\.\.\. {i}"
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(pattern, f", {result[-1]}", el)

                    pattern = f"{i}[\.]"
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(pattern, f"{i}.", el)

                    pattern = f": -{i}"
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(pattern, f": {result[-1]}", el)  

                    pattern = f"\.\. {i}" 
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(pattern, f", {result[-1]}", el)

                    pattern = f"-{i.upper()}"
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(pattern, f"{result[-1]}", el)
                el = re.sub("\. \.\.", "\.\.\.", el)

                for i in st:
                    if i == "-":
                        pattern = "|".join([f"{i}-{j}" for i in sorted(alphabet)[33:] for j in sorted(alphabet)[33:]])
                        result = re.search(pattern, el)
                        if result is not None:
                            continue
                    if i in ["'", '.', '?', '!', ',', ':', "-"]:
                        el = re.sub(f'[{i}]+', f"{i}", el)
                    elif i in ("\\", "^"):
                        el = re.sub('[\\' + f'{i}]', " ", el)   
                    else:
                        el = re.sub(f'[{i}]', " ", el)
                    el = re.sub(" +", " ", el)       
                el = re.sub(f'\? \.', f"?", el)  
                el = re.sub(f', \.', f",", el)  
                el = re.sub(f'\. \. \.', f"...", el)  
                el = re.sub(f'\.\?', f"?", el)  
                el = re.sub(f': \.', f": ", el) 
                el = re.sub('«', '\"', el)  
                el = re.sub('»', '\"', el)  
                el = re.sub(" +", " ", el) 

                for i in ("\.", "\?", ",", "!", ":"):
                    el = re.sub(f" {i}", f"{i[-1]}", el)

                el = re.sub("!\.", "!", el)  
                el = re.sub(":\.", ":", el)  
                el = re.sub("\.:", ":", el)  
                el = re.sub("!:", ":", el)  
                el = re.sub(":\.\.", ": ", el)  
                el = re.sub(" -\.", ".", el)  
                el = re.sub(",,", ",", el)  
                el = re.sub("\.,", ", ", el)  
                el = re.sub(",\.", ", ", el)  

                el = el.strip()
                if el[-1] == "-":
                    el = el[:-1] + "."
                elif el[-1].isalpha():
                    el = el + "."
                for letter in alphabet: 
                    pattern = f"{letter}{letter}{letter}"
                    result = re.search(pattern, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(f"({letter}{letter}{letter})+", f"{letter}{letter}{letter}", el)
                        
                for ban in beginnings:
                    result = re.search(ban, el)
                    if result is not None:
                        result = result.group(0)
                        el = re.sub(ban, beginnings[ban], el)
                el = re.sub(" +", " ", el) 
                
                
                text1.append(el)
            scripts[season][serie] = "\n".join(text1)
        #         if el1 != el:
        #             print(el1)
        #             print(el)
        #             print()
        else:
            continue

scripts_new = copy.deepcopy(scripts)
for season, script in scripts_new.items():
    for episode, lines in script.items():
        text = []
        for el in lines.split("\n"):
            tag = el.find(':')
            if tag > -1 and el[tag+2:] != "":
                text.append(el)
        scripts_new[season][episode] = "\n".join(text)
scripts = scripts_new 

	:  93%|█████████▎| 27/29 [00:23<00:01,  1.76it/s]


100%|██████████| 10/10 [04:22<00:00, 26.27s/it]


In [408]:
# выгружаем в локальную папку очищенные диалоги по эпизодам
try:
    os.mkdir('russian')
except:
    pass

for season, script in scripts.items():
    for episode, lines in script.items():
        try:
            os.mkdir(f'russian/{season}')
        except:
            pass
        with open(f'russian/{season}/{episode}.txt', "w", encoding="utf-8") as text_file:
            text_file.write(lines)

In [409]:
# выгружаем все реплики персонажей
script_by_character = { character : [] for character in main_characters }
script_by_character['НЕДРУГ'] = []

prev_char = 'НЕДРУГ'
prev_repl = ''

for season, script in scripts_new.items():
    for episode, lines in script.items():
        for el in lines.split("\n"):
            tag = el.find(':')
            if tag > -1:
                prev_char  = el[:tag]
                if el[tag+2:] != "" and prev_char in script_by_character:
                    script_by_character[prev_char].append(el[tag+2:])
                    
for character, script in script_by_character.items():
    with open(f'{character}.txt', "w", encoding="utf-8") as f:
        f.write(" ".join(script))

## Готовим монологи для моделей

In [21]:
# !pip install rusenttokenize

In [3]:
from rusenttokenize import ru_sent_tokenize
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
mono_by_character = { character : [] for character in main_characters }
for ch in tqdm(main_characters):
    url = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/master/data/scripts/russian/{ch}.txt'
    req = requests.get(url)
    if req.status_code == requests.codes.ok:
        req = BeautifulSoup(req.text, 'html.parser')
        text = str(req)
        for r in ru_sent_tokenize(text):
            if len(r) > 2:
                mono_by_character[ch].append(f'<s>{ch}: {r}</s>')

100%|██████████| 6/6 [00:36<00:00,  6.15s/it]


In [4]:
for ch in mono_by_character:
    res_list = mono_by_character[ch]
    random.shuffle(res_list)
    with open(f"{ch}_mono_train_9to1.txt", "w", encoding='utf-8') as text_file:
        text_file.write('\n'.join(res_list[:int(len(res_list)*0.9)]))
    with open(f"{ch}_mono_valid_9to1.txt", "w", encoding='utf-8') as text_file:
        text_file.write('\n'.join(res_list[int(len(res_list)*0.9):]))

## Чистим английские диалоги

## Чистим от биграмм

In [23]:
# чистим от лишних символов и повторяющихся нужных символов (изначальный текст)
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']

st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
      "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

script_by_serie = { season : { serie : "" for serie in range(1,30) } for season in range(1,11) }
alphabet = 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя'

for season in tqdm(range(1,11)):
    for serie in range(1, 30):
        url = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/main/data/scripts/russian/{season}/{serie}.txt'
        req = requests.get(url)
        if req.status_code == requests.codes.ok:
            req = BeautifulSoup(req.text, 'html.parser')
            text = str(req)
            text = text.split('\n')
            for el in text:
                el = delete_rep(el)
                el = delete_trash(el)
                el = re.sub("<(.*?)>", "", el)
                for i in st:
                    if i in ["'", '.', '\?', '!', ',', ':']:
                        el = re.sub(f'[{i}]+', f"{i}", el)
                    if i in ('(', ')', '*', '+', '[', '\\', '^'):
                        el = re.sub('[\\' + f'{i}]', " ", el)
                    elif i not in ["'", '.', '?', '!', ',', ':']:
                        el = re.sub(f'[{i}]', " ", el)
                    el = re.sub(" +", " ", el)
                for letter in alphabet: 
                    el = re.sub(f"({letter}{letter}{letter})+", f"{letter}", el)
                script_by_serie[season][serie] += '\n' + el
            if script_by_serie[season][serie] != '':
                with open(f"{season}_{serie}.txt", "w", encoding='utf-8') as text_file:
                    text_file.write(script_by_serie[season][serie])
        else:
            continue

100%|██████████| 10/10 [21:29<00:00, 128.94s/it]


In [15]:
script_by_character = { character : '' for character in main_characters }
script_by_character_nedrug = { character : [] for character in main_characters }
script_by_character_nedrug['НЕДРУГ'] = []
script_by_character['НЕДРУГ'] = ''

prev_char = 'НЕДРУГ'
prev_repl = ''

for season in tqdm(range(1,11)):
    for serie in range(1, 30):
        try:
            with open(f'{season}_{serie}.txt', encoding='utf-8') as f:
                text = f.readlines()
            for el in text:
                try:
                    script_by_character[el[:el.find(':')]] += ' ' + el[el.find(':')+2:].strip()
                    script_by_character_nedrug[el[:el.find(':')]].append(f'<s>НЕДРУГ: {prev_repl}\n{el.strip()}</s>')
                    prev_char  = el[:el.find(':')]
                    prev_repl = el[el.find(':')+2:].strip()
                except:
                    continue
            else:
                continue
        except:
            continue
for ch in script_by_character:
    with open(f"{ch}_clean.txt", "w", encoding='utf-8') as text_file:
        text_file.write(script_by_character[ch])
    with open(f"{ch}_clean_train.txt", "w", encoding='utf-8') as text_file:
        text_file.write('\n'.join(script_by_character_nedrug[ch][:int(len(script_by_character_nedrug[ch])*0.7)]))
    with open(f"{ch}_clean_valid.txt", "w", encoding='utf-8') as text_file:
        text_file.write('\n'.join(script_by_character_nedrug[ch][int(len(script_by_character_nedrug[ch])*0.7):]))

100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


In [ ]:
# чистим от лишних символов и повторяющихся нужных символов
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']

script_by_character = { character : [] for character in main_characters }
script_by_character['НЕДРУГ'] = []

prev_char = 'НЕДРУГ'
prev_repl = ''

st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
      "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

for season in tqdm(range(1,11)):
    for serie in range(1, 30):
        url = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/main/data/scripts/russian/{season}/{serie}.txt'
        req = requests.get(url)
        if req.status_code == requests.codes.ok:
            req = BeautifulSoup(req.text, 'html.parser')
            text = str(req)
            text = text.split('\n')
            for el in text:
                try:
                    el = re.sub("<(.*?)>", "", el)
                    for i in st:
                        if i in ["'", '.', '\?', '!', ',', ':']:
                            el = re.sub(f'[{i}]+', f"{i}", el)
                        if i in ('(', ')', '*', '+', '?', '[', '\\', '^'):
                            el = re.sub('[\\' + f'{i}]', " ", el)
                        elif i not in ["'", '.', '\?', '!', ',', ':']:
                            el = re.sub(f'[{i}]', " ", el)
                        el = re.sub(" +", " ", el)
                    script_by_character[el[:el.find(':')]].append(f'<s>НЕДРУГ: {prev_repl}\n{el.strip()}</s>')
                    prev_char  = el[:el.find(':')]
                    prev_repl = el[el.find(':')+2:].strip()
                except:
                    continue

        else:
            continue